In [1]:
import numpy as np
import datetime
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import math

In [2]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from tensorflow import keras

In [71]:
#function for scaling the input
'''
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
scaler=sc.fit(users)  #train_new[['Average_Users']]
train_scaled = scaler.transform(users)  
'''  

In [3]:
#Function to create input output for deep learning models
def create_data(n_future, n_past, train_scaled):
    trainX=[] #input
    trainY=[] #prediction
    for i in range(n_past,len(train_scaled)-n_future+1):
        trainX.append(train_scaled[i-n_past:i,0:train_scaled.shape[1]])
        trainY.append(train_scaled[i+n_future-1:i+n_future,0])
    trainX,trainY=np.array(trainX),np.array(trainY)
    return trainX, trainY

In [4]:
#function for train-test-split
#Train test split 90-10
def train_test(percentage,dataX,dataY):
    ind = int(percentage * len(dataX))
    x_tr = dataX[:ind]
    y_tr = dataY[:ind]
    x_val = dataX[ind:]
    y_val= dataY[ind:]
    return x_tr,y_tr,x_val,y_val

In [5]:
#compile
def model_compile(m,x_train,y_train,x_validate,y_validate,epoch,batch):
    m.compile(loss='mse',optimizer='adam')
    #Define the callback to save the best model during the training
    #mc = ModelCheckpoint('../out/best_model.hdf5', monitor='val_loss',verbose=1, save_best_only=True, mode='min')
    early_stopping_monitor = EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=0,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    )
    # Train the model for 30 epochs with batch size of 32:
    history=m.fit(x_train, y_train ,epochs=epoch, batch_size=batch, validation_data=(x_validate,y_validate), callbacks=[early_stopping_monitor])

In [6]:
#function to predict and inverse transform
def predict_inverse(model,train,validation_x,sc):
    forecast=model.predict(validation_x)
    y_pred_future=sc.inverse_transform(forecast)
    y_actual=train.loc[len(train)-forecast.shape[0]:,'Average_Users']
    y_pred=[float(i) for i in y_pred_future]
    y_actual=[i for i in y_actual.values]
    df = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])
    return df

In [7]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [8]:
def mse(y_true, y_pred):
    return np.mean((y_pred-y_true)**2 )

In [9]:
#Steps
#Read data, transform, create X,y
#Split into train test
#Build and compile model,
#Fit data
#Predict for validation
#compute evaluation metrics on validation data
import random
random.seed(42)
train=pd.read_csv('../out/Univariate_Wikipedia.csv')
train['Date']=pd.to_datetime(train['Date'])
train_df=train.copy()

In [10]:
train_df.head()

,Date,Average_Users
0,2015-07-01,1024.882127
1,2015-07-02,1031.233602
2,2015-07-03,973.123388
3,2015-07-04,1003.791022
4,2015-07-05,1044.341920


In [19]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
scaler=sc.fit(train_df[['Average_Users']])  #
scaled_data1 = scaler.transform(train_df[['Average_Users']]) 
X1,Y1=create_data(1, 2, scaled_data1)
x_tr1,y_tr1,x_val1,y_val1=train_test(0.90,X1,Y1)
# define model 1
model1 =  Sequential()
model1.add(LSTM(128,activation='relu',input_shape=(x_tr1.shape[1],x_tr1.shape[2]),return_sequences=True))
model1.add(Dense(64,activation='relu'))
model1.add(Dense(1,activation='linear'))
#model1.summary()
#compile model1
model_compile(model1,x_tr1,y_tr1,x_val1,y_val1,30,16)


Epoch 1/30
31/31 [==============================] - 4s 26ms/step - loss: 0.0548 - val_loss: 0.0178
Epoch 2/30
31/31 [==============================] - 0s 7ms/step - loss: 0.0172 - val_loss: 0.0160
Epoch 3/30
31/31 [==============================] - 0s 7ms/step - loss: 0.0108 - val_loss: 0.0128
Epoch 4/30
31/31 [==============================] - 0s 7ms/step - loss: 0.0073 - val_loss: 0.0104
Epoch 5/30
31/31 [==============================] - 0s 7ms/step - loss: 0.0065 - val_loss: 0.0101
Epoch 6/30
31/31 [==============================] - 0s 8ms/step - loss: 0.0064 - val_loss: 0.0103


In [20]:
model1.evaluate(x_val1,y_val1)

2/2 [==============================] - 0s 12ms/step - loss: 0.0101


0.010114029981195927

In [21]:
forecast1=model1.predict(x_val1)

2/2 [==============================] - 0s 8ms/step


In [22]:
nsamples, nx, ny = forecast1.shape
forecast = forecast1.reshape((nsamples,nx*ny))

In [23]:
y_pred_future=sc.inverse_transform(forecast)

55

In [24]:
y_actual=train_df.loc[len(train_df)-forecast1.shape[0]:,'Average_Users']
y_pred=[float(sum(i))/2 for i in y_pred_future]
y_actual=[i for i in y_actual.values]
df1 = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])

print('MAPE for LSTM = ',mape(df1['Actual'], df1['Predicted']))


MAPE for LSTM =  5.907143377972659


In [25]:

#df1.head()
print('MSE for LSTM = ',mse(df1['Actual'], df1['Predicted']))

MSE for LSTM =  15358.371555816706


In [26]:
#Stacked LSTM
# define model
sc = MinMaxScaler()
scaler=sc.fit(train_df[['Average_Users']])  #
scaled_data2 = scaler.transform(train_df[['Average_Users']]) 
X2,Y2=create_data(1, 2, scaled_data2)
x_tr2,y_tr2,x_val2,y_val2=train_test(0.90,X2,Y2)
model2 =  Sequential()
model2.add(LSTM(128,activation='relu',input_shape=(x_tr2.shape[1],x_tr2.shape[2]),return_sequences=True))
model2.add(LSTM(64,activation='relu',return_sequences=True))
model2.add(LSTM(32,activation='relu',return_sequences=False))
model2.add(Dense(32,activation='relu'))
model2.add(Dense(1,activation='linear'))


In [27]:
model2.summary()
model_compile(model2,x_tr2,y_tr2,x_val2,y_val2,30,16)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 2, 128)            66560     
                                                                 
 lstm_3 (LSTM)               (None, 2, 64)             49408     
                                                                 
 lstm_4 (LSTM)               (None, 32)                12416     
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 129473 (505.75 KB)
Trainable params: 129473 (505.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
31/

In [28]:
forecast2=model2.predict(x_val2)

2/2 [==============================] - 1s 8ms/step


In [29]:
forecast2.shape

(55, 1)

In [30]:
y_pred_future=sc.inverse_transform(forecast2)

In [31]:
y_actual=train_df.loc[len(train_df)-forecast2.shape[0]:,'Average_Users']
y_pred=[float(i) for i in y_pred_future]
y_actual=[i for i in y_actual.values]
df2 = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])

print('MAPE for Stacked LSTM = ',mape(df2['Actual'], df2['Predicted']))

MAPE for Stacked LSTM =  6.401187486266502


In [32]:
print('MSE for Stacked LSTM = ',mse(df2['Actual'], df2['Predicted']))

MSE for Stacked LSTM =  19441.96353736812


In [33]:
#GRU
sc = MinMaxScaler()
scaler=sc.fit(train_df[['Average_Users']])  #
scaled_data3 = scaler.transform(train_df[['Average_Users']]) 
X3,Y3=create_data(1, 2, scaled_data3)
x_tr3,y_tr3,x_val3,y_val3=train_test(0.90,X3,Y3)
model3=Sequential()
model3.add(GRU(128,activation='tanh',input_shape=(x_tr3.shape[1],x_tr3.shape[2]),return_sequences=True))
model3.add(Dense(64,activation='relu'))
model3.add(Dense(1,activation='linear'))
model3.summary()
model_compile(model3,x_tr3,y_tr3,x_val3,y_val3,30,16)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 2, 128)            50304     
                                                                 
 dense_6 (Dense)             (None, 2, 64)             8256      
                                                                 
 dense_7 (Dense)             (None, 2, 1)              65        
                                                                 
Total params: 58625 (229.00 KB)
Trainable params: 58625 (229.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
31/31 [==============================] - 6s 36ms/step - loss: 0.0355 - val_loss: 0.0166
Epoch 2/30
31/31 [==============================] - 0s 5ms/step - loss: 0.0083 - val_loss: 0.0100
Epoch 3/30
31/31 [==============================] - 0s 6ms/step - loss: 0.0060 - va

In [34]:
forecast3=model3.predict(x_val3)

2/2 [==============================] - 1s 0s/step


In [35]:
forecast3.shape

(55, 2, 1)

In [36]:
nsamples, nx, ny = forecast3.shape
forecastnew = forecast3.reshape((nsamples,nx*ny))

In [37]:
y_pred_future=sc.inverse_transform(forecastnew)

In [38]:
y_actual=train_df.loc[len(train_df)-forecastnew.shape[0]:,'Average_Users']
y_pred=[float(sum(i))/2 for i in y_pred_future]
y_actual=[i for i in y_actual.values]
df3 = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])

print('MAPE for GRU = ',mape(df3['Actual'], df3['Predicted']))

MAPE for GRU =  5.97025980140007


In [39]:
print('MSE for GRU = ',mse(df3['Actual'], df3['Predicted']))

MSE for GRU =  14757.255738758653


In [41]:
#Stacked GRU
sc = MinMaxScaler()
scaler=sc.fit(train_df[['Average_Users']])  #
scaled_data4 = scaler.transform(train_df[['Average_Users']]) 
X4,Y4=create_data(1, 2, scaled_data4)
x_tr4,y_tr4,x_val4,y_val4=train_test(0.90,X4,Y4)
model4=Sequential()
model4.add(GRU(128,activation='tanh',input_shape=(x_tr4.shape[1],x_tr4.shape[2]),return_sequences=True))
model4.add(GRU(64,activation='tanh',input_shape=(x_tr4.shape[1],x_tr4.shape[2]),return_sequences=True))
model4.add(GRU(32,activation='tanh',input_shape=(x_tr4.shape[1],x_tr4.shape[2]),return_sequences=False))
model4.add(Dense(64,activation='relu'))
model4.add(Dense(1,activation='linear'))


In [42]:
model4.summary()
model_compile(model4,x_tr4,y_tr4,x_val4,y_val4,30,16)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 2, 128)            50304     
                                                                 
 gru_2 (GRU)                 (None, 2, 64)             37248     
                                                                 
 gru_3 (GRU)                 (None, 32)                9408      
                                                                 
 dense_8 (Dense)             (None, 64)                2112      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 99137 (387.25 KB)
Trainable params: 99137 (387.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
31/31

In [43]:
forecast4=model4.predict(x_val4)

2/2 [==============================] - 2s 7ms/step


In [44]:
forecast4.shape

(55, 1)

In [45]:
y_pred_future=sc.inverse_transform(forecast4)

In [48]:
y_actual=train_df.loc[len(train_df)-forecast4.shape[0]:,'Average_Users']
y_pred=[float(i) for i in y_pred_future]
y_actual=[i for i in y_actual.values]
df4 = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])

print('MAPE for Stacked GRU = ',mape(df4['Actual'], df4['Predicted']))

MAPE for Stacked GRU =  6.3279812864464295


In [49]:
print('MSE for Stacked GRU = ',mse(df4['Actual'], df4['Predicted']))

MSE for Stacked GRU =  16439.137253628924


In [50]:
#Bilstm
sc = MinMaxScaler()
scaler=sc.fit(train_df[['Average_Users']])  #
scaled_data5 = scaler.transform(train_df[['Average_Users']]) 
X5,Y5=create_data(1, 2, scaled_data5)
x_tr5,y_tr5,x_val5,y_val5=train_test(0.90,X5,Y5)
model5=Sequential()
model5.add(Bidirectional(LSTM(128,activation='relu',input_shape=(x_tr5.shape[1],x_tr5.shape[2]),return_sequences=True)))
model5.add(Bidirectional(LSTM(32,activation='relu',return_sequences=False)))
model5.add(Dense(64,activation='relu'))
model5.add(Dense(1,activation='linear'))


In [51]:
#model5.summary()
model_compile(model5,x_tr5,y_tr5,x_val5,y_val5,30,16)

Epoch 1/30
31/31 [==============================] - 12s 53ms/step - loss: 0.0727 - val_loss: 0.0108
Epoch 2/30
31/31 [==============================] - 0s 11ms/step - loss: 0.0138 - val_loss: 0.0087
Epoch 3/30
31/31 [==============================] - 0s 13ms/step - loss: 0.0073 - val_loss: 0.0091


In [52]:
forecast5=model5.predict(x_val5)

2/2 [==============================] - 1s 7ms/step


In [53]:
forecast5.shape

(55, 1)

In [54]:
y_pred_future=sc.inverse_transform(forecast5)

In [55]:
y_actual=train_df.loc[len(train_df)-forecast5.shape[0]:,'Average_Users']
y_pred=[float(i) for i in y_pred_future]
y_actual=[i for i in y_actual.values]
df5 = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])

print('MAPE for BiLSTM = ',mape(df5['Actual'], df5['Predicted']))

MAPE for BiLSTM =  5.250967303795877


In [56]:
print('MSE for BiLSTM = ',mse(df5['Actual'], df5['Predicted']))

MSE for BiLSTM =  14801.56006785671


In [57]:
#CNN
sc = MinMaxScaler()
scaler=sc.fit(train_df[['Average_Users']])  #
scaled_data6 = scaler.transform(train_df[['Average_Users']]) 
X6,Y6=create_data(1, 2, scaled_data6)
x_tr6,y_tr6,x_val6,y_val6=train_test(0.90,X6,Y6)
model6 = Sequential()
model6.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_tr6.shape[1],x_tr6.shape[2])))
model6.add(MaxPooling1D(pool_size=2))
model6.add(Flatten())
model6.add(Dense(32,activation='relu'))
model6.add(Dense(1,activation='linear'))

In [58]:
model_compile(model6,x_tr6,y_tr6,x_val6,y_val6,30,16)

Epoch 1/30
31/31 [==============================] - 2s 14ms/step - loss: 0.0343 - val_loss: 0.0088
Epoch 2/30
31/31 [==============================] - 0s 5ms/step - loss: 0.0063 - val_loss: 0.0097


In [59]:
forecast6=model6.predict(x_val6)

2/2 [==============================] - 0s 0s/step


In [60]:
forecast6.shape

(55, 1)

In [61]:
y_pred_future=sc.inverse_transform(forecast6)

In [62]:
y_actual=train_df.loc[len(train_df)-forecast6.shape[0]:,'Average_Users']
y_pred=[float(i) for i in y_pred_future]
y_actual=[i for i in y_actual.values]
df6 = pd.DataFrame(list(zip(y_pred, y_actual)),columns =['Predicted', 'Actual'])

print('MAPE for CNN = ',mape(df6['Actual'], df6['Predicted']))

MAPE for CNN =  5.978940617115489


In [63]:
print('MSE for CNN = ',mse(df6['Actual'], df6['Predicted']))

MSE for CNN =  15128.013928574515
